In [175]:
from util import *
from estimator import *
import numpy as np
import random
import tensorflow as tf
slim = tf.contrib.slim
tf.reset_default_graph()

In [176]:
eps = 1e-7
batch = 10000
tau = tf.placeholder(tf.float32)
lr  = tf.placeholder(tf.float32)
x = tf.placeholder(tf.float32,[batch,784])
cc  = tf.placeholder(tf.float32,[200])
dd  = tf.placeholder(tf.float32)
kk  = tf.placeholder(tf.float32,[200])

In [177]:
def nets(estimator):
    h = slim.stack(x,slim.fully_connected,[400])
    h = slim.fully_connected(h,200,activation_fn=None)
    z = tf.nn.sigmoid(h)
    u = tf.random_uniform(tf.shape(z))
    s = tf.reshape(estimator(z,u,tau,cc,dd,kk), [-1,200])
    h2= slim.stack(slim.flatten(s),slim.fully_connected,[400])
    y = slim.fully_connected(h2,784,activation_fn=None)
    
    kld = tf.reduce_sum(z*(tf.log(z+eps)-tf.log(.5)) + (1-z)*(tf.log(1-z+eps)-tf.log(.5)), 1)
    nll = tf.nn.sigmoid_cross_entropy_with_logits(labels=x, logits=y)
    elbo= tf.reduce_sum(nll,1) + kld

    return tf.reduce_mean(elbo), elbo, tf.nn.sigmoid(y)

In [178]:
def fit_model(filename, _lr, t, c, k):
    steps = 100
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())        
        saver = tf.train.Saver()
        train_loss = np.empty((steps))
        val_loss   = np.empty((steps))    
        test_loss  = np.empty((steps))
        var_g      = np.empty((steps))
        
        d = 2.
        for i in xrange(steps*1000):
            batch_ = np.reshape(random.sample(data_train, batch), [batch,-1])
            batch_ = (np.random.uniform(0.,1.,[batch,784])<batch_).astype(float)
            _, res = sess.run([train,loss],{x:batch_, tau:t, lr:_lr, cc:c, dd:d, kk:k})
            if i%1000==1:
                var_g[i/1000] = sess.run(vg,{x:batch_, tau:t, lr:_lr, cc:c, dd:d, kk:k})
                
                if filename[:2]=='MX':
                    d = 3. - np.minimum(np.exp(-0.00001*i), 1.)
                    c = np.exp(-0.00001*i)
                    k = 1. - np.minimum(np.exp(-0.00001*i),0.1)
                else:
                    t = np.maximum(np.exp(-0.00003*i),0.1)


                train_loss[i/1000] = res
                batch_ = np.reshape(random.sample(data_val, batch), [batch,-1])
                val_loss[i/1000] = sess.run(loss,{x:batch_, tau:0.001, lr:_lr, cc:1., dd:1., kk:5.})
                batch_ = np.reshape(random.sample(data_test, batch), [batch,-1])
                test_loss[i/1000] = sess.run(loss,{x:batch_, tau:0.001, lr:_lr, cc:1., dd:1., kk:5.})
    
                print train_loss[i/1000], test_loss[i/1000], var_g[i/1000]

        np.save('VAE/OMNIGLOT/'+filename+'/loss_rec', [train_loss, val_loss, test_loss, var_g])
        save_path = saver.save(sess, 'VAE/OMNIGLOT/'+filename+"/model.ckpt")

In [179]:
data_train, data_val, data_test = bmnist()
#data_train, data_val, data_test = bomniglot()

Extracting dataset/MNIST/train-images-idx3-ubyte.gz
Extracting dataset/MNIST/train-labels-idx1-ubyte.gz
Extracting dataset/MNIST/t10k-images-idx3-ubyte.gz
Extracting dataset/MNIST/t10k-labels-idx1-ubyte.gz


In [ ]:
loss, vl, _ = nets(mse)
vg = [jacobian(vl, slim.get_model_variables()[z]) for z in range(4)]
vg = [tf.reduce_mean(tf.square(z-tf.reduce_mean(z,0))) for z in vg]
vg = tf.reduce_mean(vg)

train=tf.train.AdamOptimizer(learning_rate=lr).minimize(loss,var_list=slim.get_model_variables())
fit_model('MX1e-3', 1e-3, .1, 1., .9)

In [172]:
def gen(dn,mn):
    with tf.Session() as sess:
        saver = tf.train.Saver()
        batch_xs = data_test
        s_ = []
        s_.append(batch_xs[:100])
        for ii in mn:
            saver.restore(sess, dn+ii+"/model.ckpt")
            tl, s = sess.run([tnll,ss], {x: batch_xs})
            print tl
            s_.append(s[:100])
        return batch_xs[:100], s_

In [164]:
tnll, _, ss = nets(discrete)
a, b = gen('VAE/MNIST/',
           ['GB1e-3','ST1e-3','AST1e-3','MX1e-3_7.0'])

INFO:tensorflow:Restoring parameters from VAE/MNIST/GB1e-3/model.ckpt
107.568
INFO:tensorflow:Restoring parameters from VAE/MNIST/ST1e-3/model.ckpt
118.094
INFO:tensorflow:Restoring parameters from VAE/MNIST/AST1e-3/model.ckpt
112.013
INFO:tensorflow:Restoring parameters from VAE/MNIST/MX1e-3_7.0/model.ckpt
106.047


In [173]:
tnll, _, ss = nets(discrete)
a, b = gen('VAE/OMNIGLOT/',
           ['GB1e-3','ST3e-4','AST3e-4','MX1e-3_3.0'])

INFO:tensorflow:Restoring parameters from VAE/OMNIGLOT/GB1e-3/model.ckpt
121.863
INFO:tensorflow:Restoring parameters from VAE/OMNIGLOT/ST3e-4/model.ckpt
131.992
INFO:tensorflow:Restoring parameters from VAE/OMNIGLOT/AST3e-4/model.ckpt
122.555
INFO:tensorflow:Restoring parameters from VAE/OMNIGLOT/MX1e-3_3.0/model.ckpt
120.959


In [174]:
draw(a,b,'64',2)